In [2]:
import numpy as np
from case_studies import *
from scipy.optimize import *

In [20]:
def back_track(f,df,x,p,a=1,c1=0.01,rho=0.5):
    while f(np.array(x+a*p)) > f(np.array(x)) + c1*a* p.T@df(np.array(x)): #Check if a fulfills sufficient decrease conditions
        a = a*rho #If not, we decrease a by a factor rho
    return(a)

In [46]:
def constrained_newton(x0, A, f, df, hf,iterations=1000):
    """
    Estimates the local minimum of f using the Newton algorithm with equality constarints
    """
    x_k = x0
    for i in range(iterations):
        print(i)

        


        # Check if hessian is positive definit
        if np.all(np.linalg.eigvals(hf(x_k)) > 0):
            B = hf(x_k)
            #print("hessian",B)
        else:
            #compute approx hessian
            values,vectors = np.linalg.eig(hf(np.array(x_k)))
            for i in range(len(values)):
                B =+ np.abs(values[i]) * np.outer(vectors[i],vectors[i])

        #create the linear system
        matrix_A = np.block([[B, A.T], [A, np.zeros(A.shape[0])]])
        
        matrix_B = np.block([-df(x_k),np.zeros(A.shape[0])])

        #solve the linear system and extract pk
        solved = np.linalg.solve(matrix_A, matrix_B)

        #print(solved)
        p_k = solved[:A.shape[1]]
        #print("pk",p_k)
        lam = solved[A.shape[1]:]
        
        print("stop",np.linalg.norm(df(x_k)-A.T*lam))
        if np.linalg.norm(df(x_k)-A.T*lam) < 10**(-5):
            break

        print("pk",p_k)
        print("gradient",np.linalg.norm(df(x_k)))
        # Compute the step size
        alpha_k = back_track(x=x_k, p=p_k, f=f, df=df)
        # Update the current solution
        x_k = x_k + alpha_k*p_k

        # Check stopping condition
        if np.linalg.norm(df(x_k)) < 1e-6:
            break

        #if np.linalg.norm(lam) < stop_eps:
            #break

    return x_k,i+1


In [47]:
A = np.random.rand(1,2)
x = np.array([1,1])
b = np.array([5])

x0 = x - np.linalg.pinv(A) @ (A@x + b)

x,iterations = constrained_newton(x0=np.array(x0),A=A,f=f1,df=df1,hf=Hf1)

0
stop 1153.498777092982
pk [ 0.03504254 -0.41139587]
gradient 821.8750257048828
1
stop 29.719094414630394
pk [ 3.09031490e-16 -2.19275395e-17]
gradient 11.80457634364249
2
stop 29.719094414630415
pk [ 3.09031490e-16 -5.61458704e-20]
gradient 11.804576343642493
3
stop 29.719094414630415
pk [3.09031490e-16 5.48764321e-20]
gradient 11.804576343642493
4
stop 29.719094414630415
pk [ 3.09031490e-16 -5.61458704e-20]
gradient 11.804576343642493
5
stop 29.719094414630415
pk [3.09031490e-16 5.48764321e-20]
gradient 11.804576343642493
6
stop 29.719094414630415
pk [ 3.09031490e-16 -5.61458704e-20]
gradient 11.804576343642493
7
stop 29.719094414630415
pk [3.09031490e-16 5.48764321e-20]
gradient 11.804576343642493
8
stop 29.719094414630415
pk [ 3.09031490e-16 -5.61458704e-20]
gradient 11.804576343642493
9
stop 29.719094414630415
pk [3.09031490e-16 5.48764321e-20]
gradient 11.804576343642493
10
stop 29.719094414630415
pk [ 3.09031490e-16 -5.61458704e-20]
gradient 11.804576343642493
11
stop 29.719094

In [513]:
A = np.random.rand(1,2)
x = np.array([1,1])
b = np.array([5])

x0 = x - np.linalg.pinv(A) @ (A@x + b)

In [52]:
def constrained_steepest_descent(f,df,x,A,max_iterations=10000,epsilon=10**(-10),rho=0.5):
    """
    f is function that is being optimized
    df is the gradient of f
    x is staring point
    """
    step_length = []
    grad_norm = []

    beta = 1
    M = np.identity(A.shape[0]) - A.T @ np.linalg.inv(A@A.T) @ A
    print("M",M)
    for i in range(0,max_iterations):
        pk = -M @ df(x) #we set the ray direction
        #print("x",x)
        ak = back_track(f=f,df=df,x=x,p=pk,a=beta) #We find the step size
        #print("x",x)
        step_length.append(ak)

        print("ak",ak)

        x = x + ak*pk #we update x with the stepsize and direction

        lam = np.linalg.norm(np.linalg.inv(A@A.T)@A@df(x))
        print("stop",np.linalg.norm(df(x)-A.T*lam))
        if np.linalg.norm(df(x)-A.T*lam) < 10**(-5) :
            break
    
        beta = ak/rho #We calculate the new initial step size

        np.linalg.norm(df(x))
        grad_norm.append(np.linalg.norm(df(x)))
        #if np.linalg.norm(-pk) < epsilon: #early stopping
            #break
    return(x,i+1,step_length,grad_norm)

In [53]:
A = np.random.rand(1,2)
x = np.array([1,1])
b = np.array([5])

x0 = x - np.linalg.pinv(A) @ (A@x + b)

x,iterations,step,grad = constrained_steepest_descent(f=f1,df=df1,x=np.array(x0),A=A)

M [[0.43805292 0.50385228]
 [0.50385228 0.56194708]]
ak 0.0009765625
stop 1171.7527855426788
ak 0.0009765625
stop 247.69124810193364
ak 0.0009765625
stop 17.085017298047166
ak 0.0009765625
stop 5.631035637551808
ak 0.0009765625
stop 5.9447846048323125
ak 8.881784197001252e-16
stop 5.944784604832233
ak 8.881784197001252e-16
stop 5.944784604832153
ak 8.881784197001252e-16
stop 5.944784604832072
ak 8.881784197001252e-16
stop 5.944784604831992
ak 8.881784197001252e-16
stop 5.944784604831911
ak 8.881784197001252e-16
stop 5.944784604831831
ak 8.881784197001252e-16
stop 5.94478460483175
ak 8.881784197001252e-16
stop 5.9447846048316695
ak 8.881784197001252e-16
stop 5.9447846048315895
ak 8.881784197001252e-16
stop 5.94478460483151
ak 8.881784197001252e-16
stop 5.944784604831429
ak 8.881784197001252e-16
stop 5.944784604831348
ak 8.881784197001252e-16
stop 5.944784604831268
ak 8.881784197001252e-16
stop 5.944784604831188
ak 8.881784197001252e-16
stop 5.944784604831107
ak 8.881784197001252e-16
sto